#Scripted Primo Search Assessment Tool

Description: < Describe the purpose of the SPSAT>

## Load required libraries

In [4]:
import numpy as np
import pandas as pd
import matplotlib as plt
import glob
import os
import codecs
import json
import time
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus

##Define Some Variables

In [2]:
## URLs ##

# prod_url = <base url>
prod_url_base = 'http://bu-primo.hosted.exlibrisgroup.com:1701/PrimoWebServices/xservice/search/brief'
# stage_url = <base url>
stage_url_base = 'http://bu-primostage.hosted.exlibrisgroup.com:1701/PrimoWebServices/xservice/search/brief'

## query params ##
query_Params1 = '?institution=BOSU&query=any,contains,'
query_Params2 = '&indx=1&bulkSize=10&loc=local,scope,BOSU&loc=adaptor,primo_central_multiple_fe&onCampus=true&json=true'
stage = True
#production = True
#num_results = 10
#scope = 

## file locations ## 

# move to working directory, this is the path where we want files to reside
os.chdir('/Volumes/jwa_drive1/git/spst')

# Input file (contains search strings)
f = '/Volumes/jwa_drive1/git/spst/search_strings.csv'

##Define functions

In [3]:
# build_url 
    # takes base_url and search_string, returns url properly formated for searching
def build_url(base_url,search_string):
    url = base_url+query_Params1 + search_string.replace(' ','+') + query_Params2
    return(url)
    
# parse_response 
    # takes response (as json) from search, returns response_dict (dict of our responses)
def parse_response(response):
    response_dict = {}
    # parse the response to return the data desired elements
    response = json.loads(response.decode('utf8'))
    # need to extract the desired elements and add them to a dict
    return(response_dict)



In [5]:
# open input file, read each line as a separate string, and remove the new line character
search_strings = open(f).read().splitlines()

In [2]:
# create a DataFrame. Expand the columns to match the desired elements
# to be returned by the function 'parse_response'

# Create a dataframe with
# - column for search string 
# - column for response (returned from parse_response fuction)
df = pd.DataFrame(columns = ['SearchStr', 'Response'])

# Loop through search_strings, execute search, ... [to be done?]
for search_url in search_strings:
    time.sleep(1)
    
    
    # can we use build_url here? 
    search_url = stage_url_base+query_Params1 + search_url.replace(' ','+') + query_Params2
    request = Request(search_url)
    
    # maybe we could do ... 
    # request = Request(build_url(stage_url_base,search_url))
    
    
    try:
        response_body = urlopen(request).read() # reads request into response_body var
    
        # second dataframe? 
        df_response = pd.DataFrame([[search_url, parse_response(response_body)]],columns = ['SearchStr', 'Response'])
        df = df.append(df_response, ignore_index = True)
    #prints errors 
    except Exception as e:
        print(e)

NameError: name 'search_strings' is not defined

In [7]:
# verify dataframe
df.head()

,SearchStr,Response
0,http://bu-primostage.hosted.exlibrisgroup.com:...,{}
1,http://bu-primostage.hosted.exlibrisgroup.com:...,{}
2,http://bu-primostage.hosted.exlibrisgroup.com:...,{}
3,http://bu-primostage.hosted.exlibrisgroup.com:...,{}
4,http://bu-primostage.hosted.exlibrisgroup.com:...,{}
